# Evaluate Ensemble

This notebook combines the classification results of some models via logit-ensembling way.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

/home/imkbsz/workspace/eeg_analysis


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
import models
from train.evaluate import estimate_score
from train.evaluate import calculate_confusion_matrix
from train.evaluate import calculate_confusion_matrix2
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion, draw_confusion2
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0
cuda is available.


-----

## List up the models to check accuracy

In [4]:
model_names = [
    'lo88puq7',  # 2D-VGG-19
    'l8524nml',  # 1D-ResNet-18   // 2s1700lg, l8524nml
    'v301o425',  # 1D-ResNeXt-50 
    '1sl7ipca',  # 2D-ResNeXt-50 
    'gvqyvmrj',  # 1D-ResNet-50 
    'gjkysllw',  # 2D-ViT-B-16 
    'xci5svkl',  # 2D-ResNet-18 
    '1vc80n1f',  # 1D-VGG-19 
    'syrx7bmk',  # 2D-ResNet-50 
]

# model_names = [
#     'tp7qn5hd',  # 1D-ResNeXt-50 
#     'q1hhkmik',  # 1D-ResNet-50
#     '0svudowu',  # 2D-ResNeXt-50
#     'ruqd8r7g',  # 2D-VGG-19
#     'dn10a6bv',  # 2D-ResNet-18
#     'atbhqdgg',  # 2D-ResNet-50
#     '4439k9pg',  # 1D-ResNet-18
#     'nemy8ikm',  # 1D-VGG-19
#     '1cdws3t5',  # 2D-ViT-B-16
# ]

model_pool = []

for model_name in model_names:
    path = os.path.join(r'local/checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        print(ckpt['config']['model'])
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

2D-VGG-19
1D-ResNet-18
1D-ResNeXt-50
2D-ResNeXt-50
1D-ResNet-50
2D-ViT-B-16
2D-ResNet-18
1D-VGG-19
2D-ResNet-50
['lo88puq7',
 'l8524nml',
 'v301o425',
 '1sl7ipca',
 'gvqyvmrj',
 'gjkysllw',
 'xci5svkl',
 '1vc80n1f',
 'syrx7bmk']


---

## Configurations

In [5]:
task = 'exp-cross-dataset'
no_patient_overlap = False
eval_ensemble = True

base_repeat = 8 # 800
crop_multiple = 8
test_crop_multiple = 8

verbose = False
save_fig = True

eval_train = False
eval_val = True
eval_test = True
eval_test_multi = True

In [6]:
if no_patient_overlap:
    task += '-no-overlap'

if eval_ensemble:
    task += '-ensem'

In [7]:
@torch.no_grad()
def check_accuracy_extended(model, loader, preprocess, config, repeat=1, dummy=1):
    # for confusion matrix
    C = config["out_dims"]
    confusion_matrix = np.zeros((C, C), dtype=np.int32)

    # for ROC curve
    score = None
    target = None

    # for throughput calculation
    total = 0
    total_time = 0.0
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    # warm-up using dummy round
    for k in range(dummy):
        for sample_batched in loader:
            _ = estimate_score(model, sample_batched, preprocess, config)

    for k in range(repeat):
        for sample_batched in loader:
            # estimate
            start_event.record()
            s = estimate_score(model, sample_batched, preprocess, config)
            end_event.record()
            torch.cuda.synchronize()
            total_time += start_event.elapsed_time(end_event) / 1000

            s = s.softmax(dim=-1)
            s[:, 1] = s[:, 1] + s[:, 2]
            s = s[:, :2]
            
            y = sample_batched["class_label"]

            # classification score for drawing ROC curve
            if score is None:
                score = s.detach().cpu().numpy()
                target = y.detach().cpu().numpy()
            else:
                score = np.concatenate((score, s.detach().cpu().numpy()), axis=0)
                target = np.concatenate((target, y.detach().cpu().numpy()), axis=0)

            # confusion matrix
            pred = s.argmax(dim=-1)
            confusion_matrix += calculate_confusion_matrix(pred, y, num_classes=config["out_dims"])

            # total samples
            total += pred.shape[0]

    accuracy = confusion_matrix.trace() / confusion_matrix.sum() * 100.0
    throughput = total / total_time

    return accuracy, score, target, confusion_matrix, throughput

In [8]:
@torch.no_grad()
def check_accuracy_multicrop_extended(model, loader, preprocess, config, repeat=1, dummy=1):
    # for confusion matrix
    C = config["out_dims"]
    confusion_matrix = np.zeros((C, C), dtype=np.int32)

    # for ROC curve
    score = None
    target = None

    # for throughput calculation
    total = 0
    total_time = 0.0
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    # warm-up using dummy round
    for k in range(dummy):
        for sample_batched in loader:
            _ = estimate_score(model, sample_batched, preprocess, config)

    for k in range(repeat):
        for sample_batched in loader:
            real_minibatch = sample_batched["signal"].size(0) // config["test_crop_multiple"]
            s_merge = torch.zeros((real_minibatch, config["out_dims"]))
            y_merge = torch.zeros((real_minibatch,), dtype=torch.int32)

            # estimate
            start_event.record()
            s = estimate_score(model, sample_batched, preprocess, config)
            s = s.softmax(dim=-1)
            s[:, 1] = s[:, 1] + s[:, 2]
            s = s[:, :2]
            y = sample_batched["class_label"]
            tcm = config["test_crop_multiple"]

            # multi-crop averaging
            if s.size(0) % tcm != 0:
                raise ValueError(
                    f"check_accuracy_multicrop(): Real minibatch size={y.size(0)} is not multiple of "
                    f"config['test_crop_multiple']={tcm}."
                )

            for m in range(real_minibatch):
                s_merge[m] = s[tcm * m : tcm * (m + 1)].mean(dim=0, keepdims=True)
                y_merge[m] = y[tcm * m]

            end_event.record()
            torch.cuda.synchronize()
            total_time += start_event.elapsed_time(end_event) / 1000

            s = s_merge
            y = y_merge

            # classification score for drawing ROC curve
            if score is None:
                score = s.detach().cpu().numpy()
                target = y.detach().cpu().numpy()
            else:
                score = np.concatenate((score, s.detach().cpu().numpy()), axis=0)
                target = np.concatenate((target, y.detach().cpu().numpy()), axis=0)

            # confusion matrix
            pred = s.argmax(dim=-1)
            confusion_matrix += calculate_confusion_matrix(pred, y, num_classes=config["out_dims"])

            # total samples
            total += pred.shape[0]

    accuracy = confusion_matrix.trace() / confusion_matrix.sum() * 100.0
    throughput = total / total_time

    return accuracy, score, target, confusion_matrix, throughput


-----

## Evaluate each model and accumulate the logits

In [9]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)


    ##############################
    config['task'] = 'abnormal'
    ##############################

    
    if no_patient_overlap:
        config['task'] += '-no-overlap' 

    
    if '220419' in config['dataset_path']:
        config['dataset_path'] = './local/dataset/caueeg-dataset/'
    config['run_mode'] = 'eval'
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['photic'] = config['photic']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    model_dict['activation'] = config.get('activation', 0)

    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure and update
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_multiple'] = crop_multiple
    config['test_crop_multiple'] = test_crop_multiple
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    
    repeat = round(base_repeat / crop_multiple)
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = crop_multiple
    model_dict['test_crop_multiple'] = test_crop_multiple
    
    # build dataset
    _ = build_dataset_for_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    
    # warm-up stage
    _ = check_accuracy_extended(model, test_loader, 
                                config['preprocess_test'], config, repeat=1)
    
    # train accuracy
    if eval_train:
        _ = check_accuracy_extended(model, train_loader, 
                                    config['preprocess_test'], config, repeat=repeat)
        model_dict['Train Accuracy'] = _[0]
    
    # val accuracy
    if eval_val:
        _ = check_accuracy_extended(model, val_loader, 
                                    config['preprocess_test'], config, repeat=repeat)
        model_dict['Validation Accuracy'] = _[0]
    
    # Test accuracy
    if eval_test:
        _ = check_accuracy_extended(model, test_loader, 
                                    config['preprocess_test'], config, repeat=repeat)
        model_dict['Test Throughput'] = _[4]
        model_dict['Test Accuracy'] = _[0]
        model_dict['Test Score'] = _[1]
        model_dict['Test Target'] = _[2]
        
        pred =  model_dict['Test Score'].argmax(axis=-1)
        target = model_dict['Test Target']
        test_confusion = calculate_confusion_matrix2(pred, target, model_dict['Test Score'].shape[-1], 
                                                     len(test_loader.dataset)) 
        test_class_wise_metrics = calculate_class_wise_metrics(test_confusion.sum(axis=0))

        for k, v in test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                model_dict[f'{k} ({c_name})'] = test_class_wise_metrics[k][c]
            
        if save_fig:
            draw_roc_curve(model_dict['Test Score'], 
                           model_dict['Test Target'], 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-ROC.pdf')

            draw_confusion(test_confusion.sum(axis=0), 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-confusion.pdf')

            draw_confusion2(test_confusion.mean(axis=0), 
                            test_confusion.std(axis=0), 
                            config['class_label_to_name'], 
                            use_wandb=False, 
                            save_path=f'local/output/imgs/{model_dict["name"]}-confusion2.pdf')

            draw_class_wise_metrics(test_confusion.sum(axis=0), 
                                    config['class_label_to_name'], 
                                    use_wandb=False, 
                                    save_path=f'local/output/imgs/{model_dict["name"]}-class-wise.pdf')
    # Multi-crop test accuracy
    if eval_test_multi:
        _ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                              config['preprocess_test'], config, repeat=repeat)
        model_dict['Multi-Crop Test Throughput'] = _[4]
        model_dict['Multi-Crop Test Accuracy'] = _[0]
        model_dict['Multi-Crop Test Score'] = _[1]
        model_dict['Multi-Crop Test Target'] = _[2]
        
        pred =  model_dict['Multi-Crop Test Score'].argmax(axis=-1)
        target = model_dict['Multi-Crop Test Target']
        multi_test_confusion = calculate_confusion_matrix2(pred, target, model_dict['Test Score'].shape[-1], 
                                                           len(test_loader.dataset)) 
        multi_test_class_wise_metrics = calculate_class_wise_metrics(multi_test_confusion.sum(axis=0))

        for k, v in multi_test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                model_dict[f'Multi-Crop {k} ({c_name})'] = multi_test_class_wise_metrics[k][c]

        if save_fig:
            draw_roc_curve(model_dict['Multi-Crop Test Score'], 
                           model_dict['Multi-Crop Test Target'], 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-roc-tta.pdf')

            draw_confusion(multi_test_confusion.sum(axis=0), 
                           config['class_label_to_name'], 
                           use_wandb=False, 
                           save_path=f'local/output/imgs/{model_dict["name"]}-confusion-tta.pdf')
            
            draw_confusion2(multi_test_confusion.mean(axis=0), 
                            multi_test_confusion.std(axis=0), 
                            config['class_label_to_name'], 
                            use_wandb=False, 
                            save_path=f'local/output/imgs/{model_dict["name"]}-confusion2-tta.pdf')

            draw_class_wise_metrics(multi_test_confusion.sum(axis=0), 
                                    config['class_label_to_name'], 
                                    use_wandb=False, 
                                    save_path=f'local/output/imgs/{model_dict["name"]}-class-wise-tta.pdf') 
            
print('==== Finished ====')

- checking for lo88puq7 2D-VGG-19 ...
- checking for l8524nml 1D-ResNet-18 ...
- checking for v301o425 1D-ResNeXt-50 ...
- checking for 1sl7ipca 2D-ResNeXt-50 ...
- checking for gvqyvmrj 1D-ResNet-50 ...
- checking for gjkysllw 2D-ViT-B-16 ...
- checking for xci5svkl 2D-ResNet-18 ...
- checking for 1vc80n1f 1D-VGG-19 ...
- checking for syrx7bmk 2D-ResNet-50 ...
==== Finished ====


## Conduct ensemble

In [10]:
if eval_ensemble:
    if 'Ensemble' == model_pool[-1]['name']:
        model_pool.remove(model_pool[-1])     

    # conduct ensembling
    if eval_test:
        ensem_test_score = np.zeros_like(model_pool[0]['Test Score'])
        ensem_test_latency = 0

    if eval_test_multi:
        ensem_multi_test_score = np.zeros_like(model_pool[0]['Multi-Crop Test Score'])
        ensem_multi_test_latency = 0

    ensem_params = 0
    ensem_model_size = 0

    for model_dict in model_pool:        
        ensem_params += model_dict['num_params']
        ensem_model_size += model_dict['model size (MiB)']

        if eval_test:
            ensem_test_score += model_dict['Test Score'] / len(model_pool)
            ensem_test_latency += 1 / model_dict['Test Throughput']

        if eval_test_multi:
            ensem_multi_test_score += model_dict['Multi-Crop Test Score'] / len(model_pool)
            ensem_multi_test_latency += 1 / model_dict['Multi-Crop Test Throughput']
            
    # test accuracy
    if eval_test:
        # confusion matrix
        pred = ensem_test_score.argmax(axis=-1)
        target = model_pool[0]['Test Target']
        ensem_test_acc = 100.0 * (pred.squeeze() == target).sum() / pred.shape[0]

        ensem_test_confusion = calculate_confusion_matrix2(pred, target, ensem_test_score.shape[-1], 
                                                           len(test_loader.dataset))
        ensem_test_class_wise_metrics = calculate_class_wise_metrics(ensem_test_confusion.sum(axis=0))

        # draw
        save_path = f'local/output/imgs/{task}-ensemble-confusion.pdf' if save_fig else None
        draw_confusion(ensem_test_confusion.sum(axis=0), config['class_label_to_name'], 
                       normalize=True, use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-confusion2.pdf' if save_fig else None
        draw_confusion2(ensem_test_confusion.mean(axis=0), ensem_test_confusion.std(axis=0), 
                        config['class_label_to_name'], use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-roc.pdf' if save_fig else None
        draw_roc_curve(ensem_test_score, model_pool[0]['Test Target'], config['class_label_to_name'], 
                       use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-class-wise.pdf' if save_fig else None
        draw_class_wise_metrics(ensem_test_confusion.sum(axis=0), config['class_label_to_name'], 
                            use_wandb=False, save_path=save_path, percent=True)
        
    # multi-crop accuracy
    if eval_test_multi:
        # confusion matrix
        pred = ensem_multi_test_score.argmax(axis=-1)
        target = model_pool[0]['Multi-Crop Test Target']
        ensem_multi_test_acc = 100.0 * (pred.squeeze() == target).sum() / pred.shape[0]

        ensem_multi_test_confusion = calculate_confusion_matrix2(pred, target, ensem_multi_test_score.shape[-1], 
                                                                 len(test_loader.dataset))
        ensem_multi_test_class_wise_metrics = calculate_class_wise_metrics(ensem_multi_test_confusion.sum(axis=0))

        # draw
        save_path = f'local/output/imgs/{task}-ensemble-confusion-tta.pdf' if save_fig else None
        draw_confusion(ensem_multi_test_confusion.sum(axis=0), config['class_label_to_name'], 
                       normalize=True, use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-confusion-tta2.pdf' if save_fig else None
        draw_confusion2(ensem_multi_test_confusion.mean(axis=0), ensem_multi_test_confusion.std(axis=0), 
                        config['class_label_to_name'], use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-roc-tta.pdf' if save_fig else None
        draw_roc_curve(ensem_multi_test_score, model_pool[0]['Multi-Crop Test Target'], config['class_label_to_name'], 
                       use_wandb=False, save_path=save_path)

        save_path = f'local/output/imgs/{task}-ensemble-class-wise-tta.pdf' if save_fig else None
        draw_class_wise_metrics(ensem_multi_test_confusion.sum(axis=0), config['class_label_to_name'], 
                                use_wandb=False, save_path=save_path, percent=True)
        
    # summarize the ensemble results
    ensem_dict = {}

    ensem_dict['name'] = 'Ensemble'
    ensem_dict['num_params'] = ensem_params
    ensem_dict['model size (MiB)'] = ensem_model_size

    if eval_test:
        ensem_dict['Test Throughput'] = 1 / ensem_test_latency
        ensem_dict['Test Accuracy'] = ensem_test_acc

        for k, v in ensem_test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                ensem_dict[f'{k} ({c_name})'] = ensem_test_class_wise_metrics[k][c]

    if eval_test_multi:
        ensem_dict['Multi-Crop Test Throughput'] = 1 / ensem_multi_test_latency
        ensem_dict['Multi-Crop Test Accuracy'] = ensem_multi_test_acc

        for k, v in ensem_multi_test_class_wise_metrics.items():
            for c in range(config['out_dims']):
                c_name = config['class_label_to_name'][c]
                ensem_dict[f'Multi-Crop {k} ({c_name})'] = ensem_multi_test_class_wise_metrics[k][c]

    model_pool.append(ensem_dict)        

In [12]:
model_pool_frame = deepcopy(model_pool)

for model_dict in model_pool_frame:
    model_dict.pop('Test Score', None)
    model_dict.pop('Test Target', None)
    model_dict.pop('Multi-Crop Test Score', None)
    model_dict.pop('Multi-Crop Test Target', None)
    
pd.DataFrame(model_pool_frame).to_csv(f'local/output/{task}.csv')
pd.DataFrame(model_pool_frame)

,name,path,model,num_params,model size (MiB),seq_length,use_age,photic,EKG,awgn,...,Multi-Crop Accuracy (Normal),Multi-Crop Accuracy (Abnormal),Multi-Crop Sensitivity (Normal),Multi-Crop Sensitivity (Abnormal),Multi-Crop Specificity (Normal),Multi-Crop Specificity (Abnormal),Multi-Crop Precision (Normal),Multi-Crop Precision (Abnormal),Multi-Crop F1-score (Normal),Multi-Crop F1-score (Abnormal)
0,lo88puq7,local/checkpoint/lo88puq7/checkpoint.pt,2D-VGG-19,20184131,77.073947,4000.0,conv,X,O,0.047940,...,0.691176,0.691176,0.826087,0.622222,0.622222,0.826087,0.527778,0.875000,0.644068,0.727273
1,l8524nml,local/checkpoint/l8524nml/checkpoint.pt,1D-ResNet-18,11394051,43.551189,2000.0,conv,O,O,0.004873,...,0.698529,0.698529,0.760870,0.666667,0.666667,0.760870,0.538462,0.845070,0.630631,0.745342
2,v301o425,local/checkpoint/v301o425/checkpoint.pt,1D-ResNeXt-50,25650051,98.228438,4000.0,fc,X,O,0.037536,...,0.808824,0.808824,0.717391,0.855556,0.855556,0.717391,0.717391,0.855556,0.717391,0.855556
3,1sl7ipca,local/checkpoint/1sl7ipca/checkpoint.pt,2D-ResNeXt-50,25886467,99.137356,4000.0,fc,O,X,0.103950,...,0.742647,0.742647,0.869565,0.677778,0.677778,0.869565,0.579710,0.910448,0.695652,0.777070
4,gvqyvmrj,local/checkpoint/gvqyvmrj/checkpoint.pt,1D-ResNet-50,26178179,100.185495,1000.0,fc,O,X,0.012513,...,0.639706,0.639706,0.760870,0.577778,0.577778,0.760870,0.479452,0.825397,0.588235,0.679739
5,gjkysllw,local/checkpoint/gjkysllw/checkpoint.pt,2D-ViT-B-16,90054147,343.588083,12000.0,conv,X,O,0.029602,...,0.691176,0.691176,0.782609,0.644444,0.644444,0.782609,0.529412,0.852941,0.631579,0.734177
6,xci5svkl,local/checkpoint/xci5svkl/checkpoint.pt,2D-ResNet-18,11425155,43.664536,2000.0,fc,X,O,0.074615,...,0.683824,0.683824,0.760870,0.644444,0.644444,0.760870,0.522388,0.840580,0.619469,0.729560
7,1vc80n1f,local/checkpoint/1vc80n1f/checkpoint.pt,1D-VGG-19,20205251,77.157321,2000.0,conv,X,O,0.008120,...,0.727941,0.727941,0.739130,0.722222,0.722222,0.739130,0.576271,0.844156,0.647619,0.778443
8,syrx7bmk,local/checkpoint/syrx7bmk/checkpoint.pt,2D-ResNet-50,25729475,98.468181,4000.0,conv,X,O,0.002375,...,0.647059,0.647059,0.717391,0.611111,0.611111,0.717391,0.485294,0.808824,0.578947,0.696203
9,Ensemble,NaN,NaN,256706907,981.054547,NaN,NaN,NaN,NaN,NaN,...,0.713235,0.713235,0.804348,0.666667,0.666667,0.804348,0.552239,0.869565,0.654867,0.754717


##### 

##### 